In [14]:
import qrcode
import pandas as pd
from PIL import Image, ImageDraw, ImageFont
import io
import emoji

def es_emoji(texto):
    return emoji.is_emoji(texto)

def crear_qr(url):
    if pd.isna(url) or url == '':
        qr = qrcode.QRCode(box_size=10)
        qr.add_data("No disponible")
        return qr.make_image()
    
    # Si la URL no comienza con http:// o https://, asumimos que es un nombre para buscar
    if not url.startswith(('http://', 'https://')):
        # Crear URL de búsqueda de Google
        url = f"https://www.google.com/search?q={url}"
    
    qr = qrcode.QRCode(box_size=10)
    qr.add_data(url)
    return qr.make_image()

def procesar_entrada(row):
    # Crear una imagen en blanco para el layout
    ancho_total = 800
    alto_total = 400
    imagen_final = Image.new('RGB', (ancho_total, alto_total), 'white')
    draw = ImageDraw.Draw(imagen_final)
    
    # Configurar la fuente (necesitarás una fuente TTF)
    try:
        fuente = ImageFont.truetype("Arial.ttf", 30)
    except:
        fuente = ImageFont.load_default()
    
    # Procesar las etiquetas
    etiquetas = row['#género #disciplina'].split()
    etiquetas_formateadas = []
    
    for etiqueta in etiquetas:
        if es_emoji(etiqueta):
            etiquetas_formateadas.append(etiqueta)  # Mantener emoji sin #
        else:
            etiquetas_formateadas.append(f"#{etiqueta.lstrip('#')}")  # Añadir # a texto
    
    etiquetas_final = ' '.join(etiquetas_formateadas)
    
    # Escribir el título (Nombre: género/disciplina)
    titulo = f"{row['NOMBRE ARTÍSTICO']}: {etiquetas_final}"
    draw.text((20, 20), titulo, fill='black', font=fuente)
    
    # Generar los dos QR
    qr1 = crear_qr(row['LINK_WEB_1'])
    qr2 = crear_qr(row['LINK_WEB_2'])
    
    # Redimensionar los QR si es necesario
    tamano_qr = 300
    qr1 = qr1.resize((tamano_qr, tamano_qr))
    qr2 = qr2.resize((tamano_qr, tamano_qr))
    
    # Pegar los QR en la imagen final
    imagen_final.paste(qr1, (50, 80))
    imagen_final.paste(qr2, (450, 80))
    
    return imagen_final

# Leer el CSV
df = pd.read_csv('data/gen.csv')

# Procesar cada fila y guardar las imágenes
for index, row in df.iterrows():
    imagen = procesar_entrada(row)
    imagen.save(f"qr_layout_{row['NOMBRE ARTÍSTICO']}.png")